# CrewAI with Couchbase Short-Term Memory using GSI Vector Search

This notebook demonstrates how to implement a custom storage backend for CrewAI's memory system using Couchbase and GSI (Global Secondary Index) vector search. GSI vector search provides high-performance semantic search capabilities and is available in Couchbase Server 8.0.0+. Alternatively if you want to perform semantic search using the FTS index, please take a look at [this.](../fts/CouchbaseStorage_Demo.ipynb)

Here's a breakdown of each section:

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run 
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

**Important**: GSI vector search requires Couchbase Server 8.0.0+ or Couchbase Capella with Query Service enabled.

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy 
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn 
     about Capella with no time constraint
   - To learn more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the required bucket (Read and Write) 
     used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)
   - Ensure Query Service is enabled on your cluster for GSI vector search functionality

# Memory in AI Agents

Memory in AI agents is a crucial capability that allows them to retain and utilize information across interactions, making them more effective and contextually aware. Without memory, agents would be limited to processing only the immediate input, lacking the ability to build upon past experiences or maintain continuity in conversations.

> Note: This section on memory types and functionality is adapted from the CrewAI documentation.

## Types of Memory in AI Agents

### Short-term Memory
- Retains recent interactions and context
- Typically spans the current conversation or session  
- Helps maintain coherence within a single interaction flow
- In CrewAI, this is what we're implementing with the Couchbase storage

### Long-term Memory
- Stores persistent knowledge across multiple sessions
- Enables agents to recall past interactions even after long periods
- Helps build cumulative knowledge about users, preferences, and past decisions
- While this implementation is labeled as "short-term memory", the Couchbase storage backend can be effectively used for long-term memory as well, thanks to Couchbase's persistent storage capabilities and enterprise-grade durability features



## How Memory Works in Agents
Memory in AI agents typically involves:
- Storage: Information is encoded and stored in a database (like Couchbase, ChromaDB, or other vector stores)
- Retrieval: Relevant memories are fetched based on semantic similarity to current context
- Integration: Retrieved memories are incorporated into the agent's reasoning process

In the CrewAI example, the CouchbaseStorage class implements:
- save(): Stores new memories with metadata
- search(): Retrieves relevant memories based on semantic similarity
- reset(): Clears stored memories when needed

## Benefits of Memory in AI Agents
- Contextual Understanding: Agents can refer to previous parts of a conversation
- Personalization: Remembering user preferences and past interactions
- Learning and Adaptation: Building knowledge over time to improve responses
- Task Continuity: Resuming complex tasks across multiple interactions
- Collaboration: In multi-agent systems like CrewAI, memory enables agents to build on each other's work

## Memory in CrewAI Specifically
In CrewAI, memory serves several important functions:
- Agent Specialization: Each agent can maintain its own memory relevant to its expertise
- Knowledge Transfer: Agents can share insights through memory when collaborating on tasks
- Process Continuity: In sequential processes, later agents can access the work of earlier agents
- Contextual Awareness: Agents can reference previous findings when making decisions

The vector-based approach (using embeddings) is particularly powerful because it allows for semantic search - finding memories that are conceptually related to the current context, not just exact keyword matches.

By implementing custom storage like Couchbase, you gain additional benefits like persistence, scalability, and the ability to leverage enterprise-grade database features for your agent memory systems.

## Install Required Libraries

This section installs the necessary Python packages:
- `crewai`: The main CrewAI framework
- `langchain-couchbase`: LangChain integration for Couchbase
- `langchain-openai`: LangChain integration for OpenAI
- `python-dotenv`: For loading environment variables

In [1]:
%pip install --quiet crewai==0.186.1 langchain-couchbase==0.5.0rc1 langchain-openai==0.3.33 python-dotenv==1.1.1

Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries

The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
from typing import Any, Dict, List, Optional
import os
import logging
from datetime import timedelta
from dotenv import load_dotenv
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Agent, Crew, Task, Process
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.diagnostics import PingState, ServiceType
from langchain_couchbase.vectorstores import CouchbaseQueryVectorStore
from langchain_couchbase.vectorstores import DistanceStrategy
from langchain_couchbase.vectorstores import IndexType
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import time
import json
import uuid

# Configure logging (disabled)
logging.basicConfig(level=logging.CRITICAL)
logger = logging.getLogger(__name__)

## Loading Sensitive Information

In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

### Setting Up Environment Variables

> **Note:** This implementation reads configuration parameters from environment variables. Before running this notebook, you need to set the following environment variables:
>
> - `OPENAI_API_KEY`: Your OpenAI API key for generating embeddings
> - `CB_HOST`: Couchbase cluster connection string (e.g., "couchbases://cb.example.com")
> - `CB_USERNAME`: Username for Couchbase authentication
> - `CB_PASSWORD`: Password for Couchbase authentication
> - `CB_BUCKET_NAME` (optional): Bucket name (defaults to "vector-search-testing")
> - `SCOPE_NAME` (optional): Scope name (defaults to "shared")
> - `COLLECTION_NAME` (optional): Collection name (defaults to "crew")
> - `INDEX_NAME` (optional): GSI vector index name (defaults to "vector_search_crew")
>
> You can set these variables in a `.env` file in the same directory as this notebook, or set them directly in your environment.

In [3]:
load_dotenv("./.env")

# Verify environment variables
required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD']
for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"{var} environment variable is required")

## Understanding GSI Vector Search Types

Couchbase offers two types of GSI vector indexes for different use cases:

Hyperscale Vector Indexes (BHIVE)
- Best for pure vector searches - content discovery, recommendations, semantic search
- High performance with low memory footprint - designed to scale to billions of vectors
- Optimized for concurrent operations - supports simultaneous searches and inserts
- Use when: You primarily perform vector-only queries without complex scalar filtering
- Ideal for: Large-scale semantic search, recommendation systems, content discovery

Composite Vector Indexes 
- Best for filtered vector searches - combines vector search with scalar value filtering
- Efficient pre-filtering - scalar attributes reduce the vector comparison scope
- Use when: Your queries combine vector similarity with scalar filters that eliminate large portions of data
- Ideal for: Compliance-based filtering, user-specific searches, time-bounded queries

Choosing the Right Index Type
- Start with Hyperscale Vector Index for pure vector searches and large datasets
- Use Composite Vector Index when scalar filters significantly reduce your search space
- Consider your dataset size: Hyperscale scales to billions, Composite works well for tens of millions to billions

For this CrewAI memory implementation, we'll use **BHIVE** as it's optimized for pure semantic search scenarios typical in AI agent memory systems.

## Understanding Index Configuration (Couchbase 8.0 Feature)

The index_description parameter controls how Couchbase optimizes vector storage and search performance through centroids and quantization:

Format: `'IVF[<centroids>],{PQ|SQ}<settings>'`

Centroids (IVF - Inverted File):
- Controls how the dataset is subdivided for faster searches
- More centroids = faster search, slower training  
- Fewer centroids = slower search, faster training
- If omitted (like IVF,SQ8), Couchbase auto-selects based on dataset size

Quantization Options:
- SQ (Scalar Quantization): SQ4, SQ6, SQ8 (4, 6, or 8 bits per dimension)
- PQ (Product Quantization): PQ<subquantizers>x<bits> (e.g., PQ32x8)
- Higher values = better accuracy, larger index size

Common Examples:
- IVF,SQ8 - Auto centroids, 8-bit scalar quantization (good default)
- IVF1000,SQ6 - 1000 centroids, 6-bit scalar quantization  
- IVF,PQ32x8 - Auto centroids, 32 subquantizers with 8 bits

For detailed configuration options, see the [Quantization & Centroid Settings](https://preview.docs-test.couchbase.com/docs-server-DOC-12565_vector_search_concepts/server/current/vector-index/hyperscale-vector-index.html#algo_settings).


## Implement CouchbaseStorage with GSI Vector Search

This section demonstrates the implementation of a custom vector storage solution using Couchbase GSI vector search:

> **Note on Implementation:** This example uses the LangChain Couchbase integration (`langchain_couchbase`) with `CouchbaseQueryVectorStore` for GSI-based vector search. This provides high-performance semantic search capabilities using Couchbase's Query Service and GSI vector indexes.

> **GSI Requirements:** GSI vector search requires Couchbase Server 8.0.0+ or Couchbase Capella with Query Service enabled.

> For more information on GSI vector search, refer to:
> - [Couchbase GSI Vector Search Documentation](https://docs.couchbase.com/server/current/vector-index/use-vector-indexes.html)

In [4]:
class CouchbaseStorage(RAGStorage):
    """
    Extends RAGStorage to handle embeddings for memory entries using Couchbase GSI Vector Search.
    """

    def __init__(self, type: str, allow_reset: bool = True, embedder_config: Optional[Dict[str, Any]] = None, crew: Optional[Any] = None):
        """Initialize CouchbaseStorage with GSI vector search configuration."""
        super().__init__(type, allow_reset, embedder_config, crew)
        self._initialize_app()

    def search(
        self,
        query: str,
        limit: int = 3,
        filter: Optional[dict] = None,
        score_threshold: float = 0,
    ) -> List[Dict[str, Any]]:
        """
        Search memory entries using GSI vector similarity.
        """
        try:
            # Add type filter
            search_filter = {"memory_type": self.type}
            if filter:
                search_filter.update(filter)

            # Execute search using GSI vector search
            results = self.vector_store.similarity_search_with_score(
                query,
                k=limit,
                filter=search_filter
            )
            
            # Format results and deduplicate by content
            seen_contents = set()
            formatted_results = []
            
            for i, (doc, score) in enumerate(results):
                # Note: In GSI vector search, lower scores indicate higher similarity
                if score <= (1.0 - score_threshold):  # Convert threshold for GSI distance metric
                    content = doc.page_content
                    if content not in seen_contents:
                        seen_contents.add(content)
                        formatted_results.append({
                            "id": doc.metadata.get("memory_id", str(i)),
                            "metadata": doc.metadata,
                            "context": content,
                            "score": float(score)
                        })
            
            logger.info(f"Found {len(formatted_results)} unique results for query: {query}")
            return formatted_results

        except Exception as e:
            logger.error(f"Search failed: {str(e)}")
            return []

    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        """
        Save a memory entry with metadata.
        """
        try:
            # Generate unique ID
            memory_id = str(uuid.uuid4())
            timestamp = int(time.time() * 1000)
            
            # Prepare metadata (create a copy to avoid modifying references)
            if not metadata:
                metadata = {}
            else:
                metadata = metadata.copy()  # Create a copy to avoid modifying references
                
            # Process agent-specific information if present
            agent_name = metadata.get('agent', 'unknown')
                
            # Clean up value if it has the typical LLM response format
            value_str = str(value)
            if "Final Answer:" in value_str:
                # Extract just the actual content - everything after "Final Answer:"
                parts = value_str.split("Final Answer:", 1)
                if len(parts) > 1:
                    value = parts[1].strip()
                    logger.info(f"Cleaned up response format for agent: {agent_name}")
            elif value_str.startswith("Thought:"):
                # Handle thought/final answer format
                if "Final Answer:" in value_str:
                    parts = value_str.split("Final Answer:", 1)
                    if len(parts) > 1:
                        value = parts[1].strip()
                        logger.info(f"Cleaned up thought process format for agent: {agent_name}")
            
            # Update metadata
            metadata.update({
                "memory_id": memory_id,
                "memory_type": self.type,
                "timestamp": timestamp,
                "source": "crewai"
            })

            # Log memory information for debugging
            value_preview = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
            metadata_preview = {k: v for k, v in metadata.items() if k != "embedding"}
            logger.info(f"Saving memory for Agent: {agent_name}")
            logger.info(f"Memory value preview: {value_preview}")
            logger.info(f"Memory metadata: {metadata_preview}")
            
            # Convert value to string if needed
            if isinstance(value, (dict, list)):
                value = json.dumps(value)
            elif not isinstance(value, str):
                value = str(value)

            # Save to GSI vector store
            self.vector_store.add_texts(
                texts=[value],
                metadatas=[metadata],
                ids=[memory_id]
            )
            logger.info(f"Saved memory {memory_id}: {value[:100]}...")

        except Exception as e:
            logger.error(f"Save failed: {str(e)}")
            raise

    def reset(self) -> None:
        """Reset the memory storage if allowed."""
        if not self.allow_reset:
            return

        try:
            # Delete documents of this memory type
            self.cluster.query(
                f"DELETE FROM `{self.bucket_name}`.`{self.scope_name}`.`{self.collection_name}` WHERE memory_type = $type",
                type=self.type
            ).execute()
            logger.info(f"Reset memory type: {self.type}")
        except Exception as e:
            logger.error(f"Reset failed: {str(e)}")
            raise

    def _initialize_app(self):
        """Initialize Couchbase connection and GSI vector store."""
        try:
            # Initialize embeddings
            if self.embedder_config and self.embedder_config.get("provider") == "openai":
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model=self.embedder_config.get("config", {}).get("model", "text-embedding-3-small")
                )
            else:
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model="text-embedding-3-small"
                )

            # Connect to Couchbase
            auth = PasswordAuthenticator(
                os.getenv('CB_USERNAME', ''),
                os.getenv('CB_PASSWORD', '')
            )
            options = ClusterOptions(auth)
            
            # Initialize cluster connection
            self.cluster = Cluster(os.getenv('CB_HOST', ''), options)
            self.cluster.wait_until_ready(timedelta(seconds=5))

            # Check Query service (required for GSI vector search)
            ping_result = self.cluster.ping()
            query_available = False
            for service_type, endpoints in ping_result.endpoints.items():
                if service_type.name == 'Query':  # Query Service for GSI
                    for endpoint in endpoints:
                        if endpoint.state == PingState.OK:
                            query_available = True
                            logger.info(f"Query service is responding at: {endpoint.remote}")
                            break
                    break
            if not query_available:
                raise RuntimeError("Query service not found or not responding. GSI vector search requires Query Service.")
            
            # Set up storage configuration
            self.bucket_name = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
            self.scope_name = os.getenv('SCOPE_NAME', 'shared')
            self.collection_name = os.getenv('COLLECTION_NAME', 'crew')
            self.index_name = os.getenv('INDEX_NAME', 'vector_search_crew_gsi')

            # Initialize GSI vector store
            self.vector_store = CouchbaseQueryVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=self.scope_name,
                collection_name=self.collection_name,
                embedding=self.embeddings,
                distance_metric=DistanceStrategy.COSINE,
            )
            logger.info(f"Initialized CouchbaseStorage with GSI vector search for type: {self.type}")

        except Exception as e:
            logger.error(f"Initialization failed: {str(e)}")
            raise


## Test Basic Storage

Test storing and retrieving a simple memory:

In [5]:
# Initialize storage
storage = CouchbaseStorage(
    type="short_term",
    embedder_config={
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"}
    }
)

# Reset storage
storage.reset()

# Test storage
test_memory = "Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'"
test_metadata = {"category": "sports", "test": "initial_memory"}
storage.save(test_memory, test_metadata)

# Test search
results = storage.search("What did Guardiola say about Manchester City?", limit=1)
for result in results:
    print(f"Found: {result['context']}\nScore: {result['score']}\nMetadata: {result['metadata']}")

Found: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'
Score: 0.3401162643249742
Metadata: {'category': 'sports', 'memory_id': 'fb094c4d-a091-43e9-b7b4-647688640e00', 'memory_type': 'short_term', 'source': 'crewai', 'test': 'initial_memory', 'timestamp': 1758093766061}


## Create GSI Vector Index

Before we can use the GSI vector store for searching, we need to create a GSI vector index. This step is crucial for GSI vector search to work properly.


In [6]:
# Create GSI BHIVE vector index for optimal performance
try:
    storage.vector_store.create_index(
        index_type=IndexType.BHIVE,
        index_name=storage.index_name,
        index_description="IVF,SQ8"  # Auto-selected centroids with 8-bit scalar quantization
    )
    print(f"Created GSI BHIVE vector index: {storage.index_name}")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"GSI vector index '{storage.index_name}' already exists, continuing...")
    else:
        print(f"Error creating GSI index: {str(e)}")


GSI vector index 'vector_search_crew' already exists, continuing...


## Test CrewAI Integration

Create agents and tasks to test memory retention:

In [7]:
# Initialize ShortTermMemory with our storage
memory = ShortTermMemory(storage=storage)

# Initialize language model
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)

# Create agents with memory
sports_analyst = Agent(
    role='Sports Analyst',
    goal='Analyze Manchester City performance',
    backstory='Expert at analyzing football teams and providing insights on their performance',
    llm=llm,
    memory=True,
    memory_storage=memory
)

journalist = Agent(
    role='Sports Journalist',
    goal='Create engaging football articles',
    backstory='Experienced sports journalist who specializes in Premier League coverage',
    llm=llm,
    memory=True,
    memory_storage=memory
)

# Create tasks
analysis_task = Task(
    description='Analyze Manchester City\'s recent performance based on Pep Guardiola\'s comments: "The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased."',
    agent=sports_analyst,
    expected_output="A comprehensive analysis of Manchester City's current form based on Guardiola's comments."
)

writing_task = Task(
    description='Write a sports article about Manchester City\'s form using the analysis and Guardiola\'s comments.',
    agent=journalist,
    context=[analysis_task],
    expected_output="An engaging sports article about Manchester City's current form and Guardiola's perspective."
)

# Create crew with memory
crew = Crew(
    agents=[sports_analyst, journalist],
    tasks=[analysis_task, writing_task],
    process=Process.sequential,
    memory=True,
    short_term_memory=memory,  # Explicitly pass our memory implementation
    verbose=True
)

# Run the crew
result = crew.kickoff()

print("\nCrew Result:")
print("-" * 80)
print(result)
print("-" * 80)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cdf47a13-444b-4a9b-94c7-f5d7e654b636                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/viraj.agarwal/Tasks/Task10/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Provide detailed statistics or metrics to strengthen the analysis, such as recent match outcomes, player     │
│  performances, or league standings.                                                                             │
│  - Include direct quotes from Pep Guardiola or other credible sources to enhance the argument.                  │
│  - Mention potential challenges or areas of improvement for the team despite the positive remarks.              │
│  - Analyze the comments in the context of the team's performance against different opponents or in specific     │
│  competitions.                                                                                                  │
│  - Include historical cont...                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 1262.32ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sports Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze Manchester City's recent performance based on Pep Guardiola's comments: "The team is playing     │
│  well, we are in a good moment. The way we are training, the way we are playing - I am really pleased."         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sports Analyst                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Manchester City's recent performance under Pep Guardiola's management has been notably impressive, as          │
│  highlighted by Guardiola's own words: "The team is playing well, we are in a good moment. The way we are       │
│  training, the way we are playing - I am really pleased." This statement encapsulates the essence of City's     │
│  current success, which stems from a harmonious blend of strategic execution, individual brilliance, and        │
│  collective resilience.                                                                                         │
│                                                                                                                 │
│  Guardiola's satisfaction with the team's current form indicates that Manchester City is exhibiting a high      │
│  level of play both in matches and during training sessions. This reflects the players' strong work ethic and   │
│  the coaching staff's effective planning and execution. Training well is crucial as it lays the foundation for  │
│  match day performances, suggesting that the team is well-prepared physically and tactically.                   │
│                                                                                                                 │
│  Historically, under Guardiola's tenure, Manchester City has set high standards in terms of performance, and    │
│  the current form appears to be in line with those standards. The players have shown consistency in executing   │
│  their game plans, which is paramount in a league as competitive as the Premier League. City's ability to       │
│  adapt and overcome unique challenges posed by different opponents is a clear indicator of their preparedness   │
│  and tactical flexibility.                                                                                      │
│                                                                                                                 │
│  In recent matches, City has demonstrated their dominance with solid results, maintaining a competitive edge    │
│  in their competitions. The cohesion and understanding displayed by the players suggest a team that is not      │
│  only well-drilled but also mentally attuned to the demands of top-flight football. This level of performance   │
│  is often characterized by meticulous preparation, which has become synonymous with Guardiola's management      │
│  style.                                                                                                         │
│                                                                                                                 │
│  Moreover, the current form reflects a period of stability and success for Manchester City, with the team       │
│  likely achieving their set targets with remarkable efficiency. Guardiola's leadership has fostered a culture   │
│  of excellence, where every player is committed to the cause, and every match is approached with the same       │
│  intensity. This positive environment is crucial as City continues to compete across various tournaments.       │
│                                                                                                                 │
│  Despite the positive remarks, potential challenges remain, such as maintaining this level of performance       │
│  consistently throughout the season and managing the ph

...

/Users/viraj.agarwal/Tasks/Task10/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/viraj.agarwal/Tasks/Task10/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9c9865ac-3f38-4324-b912-d470565be9b1                                                                     │
│  Agent: Sports Journalist                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cdf47a13-444b-4a9b-94c7-f5d7e654b636                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Manchester City: Riding High on a Wave of Form and Confidence**                                │
│                                                                                                                 │
│  Manchester City is currently riding the crest of a wave, showcasing a remarkable run of form that has not      │
│  gone unnoticed by their astute manager, Pep Guardiola. With a string of positive performances, both in         │
│  matches and on the training ground, City is demonstrating a level of play that is befitting their stature as   │
│  one of the Premier League's most formidable forces.                                                            │
│                                                                                                                 │
│  In a recent press conference, Guardiola couldn't hide his satisfaction, stating, "The team is playing well,    │
│  we are in a good moment. The way we are training, the way we are playing - I am really pleased." These words   │
│  encapsulate the essence of City's current success - a harmonious blend of strategic execution, individual      │
│  brilliance, and collective resilience.                                                                         │
│                                                                                                                 │
│  Manchester City's recent performances have been a testament to their rigorous and well-structured training     │
│  sessions. Guardiola's emphasis on the importance of training highlights the meticulous preparation that has    │
│  become synonymous with his management style. It's this attention to detail that ensures City not only          │
│  competes but often dominates their opponents.                                                                  │
│                                                                                                                 │
│  The players' strong work ethic is evident on the pitch, where they have shown consistency in executing their   │
│  game plans. This consistency is crucial, especially in a league as competitive as the Premier League, where    │
│  every match can present a unique challenge. City's ability to adapt and overcome these challenges is a clear   │
│  indicator of their preparedness and tactical flexibility.                                                      │
│                                                                                                                 │
│  Moreover, the current form of Manchester City reflects a period of stability and success that has them         │
│  ticking off their targets with remarkable efficiency. Guardiola's satisfaction is not just rooted in the       │
│  results; it's about how those results are achieved. The cohesion and understanding displayed by the players    │
│  suggest a team that is not only well-drilled but also mentally attuned to the demands of top-flight football.  │
│                                                                                                                 │
│  As City continues to compete across various tournaments, this positive environment is crucial. The confidence  │
│  built during this run can often be the difference in 


Crew Result:
--------------------------------------------------------------------------------
**Manchester City: Riding High on a Wave of Form and Confidence**

Manchester City is currently riding the crest of a wave, showcasing a remarkable run of form that has not gone unnoticed by their astute manager, Pep Guardiola. With a string of positive performances, both in matches and on the training ground, City is demonstrating a level of play that is befitting their stature as one of the Premier League's most formidable forces.

In a recent press conference, Guardiola couldn't hide his satisfaction, stating, "The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased." These words encapsulate the essence of City's current success - a harmonious blend of strategic execution, individual brilliance, and collective resilience.

Manchester City's recent performances have been a testament to their rigorous and well-structured traini

## Test Memory Retention

Query the stored memories to verify retention:

In [8]:
# Wait for memories to be stored
time.sleep(2)

# List all documents in the collection
try:
    # Query to fetch all documents of this memory type
    query_str = f"SELECT META().id, * FROM `{storage.bucket_name}`.`{storage.scope_name}`.`{storage.collection_name}` WHERE memory_type = $type"
    query_result = storage.cluster.query(query_str, type=storage.type)
    
    print(f"\nAll memory entries in Couchbase:")
    print("-" * 80)
    for i, row in enumerate(query_result, 1):
        doc_id = row.get('id')
        memory_id = row.get(storage.collection_name, {}).get('memory_id', 'unknown')
        content = row.get(storage.collection_name, {}).get('text', '')[:100] + "..."  # Truncate for readability
        
        print(f"Entry {i}:")
        print(f"ID: {doc_id}")
        print(f"Memory ID: {memory_id}")
        print(f"Content: {content}")
        print("-" * 80)
except Exception as e:
    print(f"Failed to list memory entries: {str(e)}")

# Test memory retention
memory_query = "What is Manchester City's current form according to Guardiola?"
memory_results = storage.search(
    query=memory_query,
    limit=5,  # Increased to see more results
    score_threshold=0.0  # Lower threshold to see all results
)

print("\nMemory Search Results:")
print("-" * 80)
for result in memory_results:
    print(f"Context: {result['context']}")
    print(f"Score: {result['score']}")
    print("-" * 80)

# Try a more specific query to find agent interactions
interaction_query = "Manchester City playing style analysis tactical"
interaction_results = storage.search(
    query=interaction_query,
    limit=5,
    score_threshold=0.0
)

print("\nAgent Interaction Memory Results:")
print("-" * 80)
for result in interaction_results:
    print(f"Context: {result['context'][:200]}...")  # Limit output size
    print(f"Score: {result['score']}")
    print("-" * 80)



All memory entries in Couchbase:
--------------------------------------------------------------------------------

Memory Search Results:
--------------------------------------------------------------------------------
Context: Pep Guardiola praised Manchester City's current form, saying 'The team is playing well, we are in a good moment. The way we are training, the way we are playing - I am really pleased.'
Score: 0.285379886892123
--------------------------------------------------------------------------------
Context: Manchester City's recent performance has been notably positive, as highlighted by Pep Guardiola's comments. Guardiola's satisfaction with the team's current form suggests that City is exhibiting a high level of play, both in matches and in training sessions. The statement "The team is playing well, we are in a good moment" indicates that the players are executing strategies effectively and maintaining consistency in their performances. This likely translates to solid